In [1]:
import pandas as pd
from preprocess import preprocess_text
import numpy as np

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tirth\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\tirth\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to punkt...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
labelled_data = pd.read_csv("D://FALL 2022//CSC2515 INTRO TO ML//Active_Learning_Based_Research_Paper_Classification//data//labelled//labelled_data.csv")

In [3]:
unlabelled_data = pd.read_csv("D://FALL 2022//CSC2515 INTRO TO ML//Active_Learning_Based_Research_Paper_Classification//data//unlabelled_data//unlabelled_data_json.csv")

In [4]:
df = labelled_data[['title', 'abstract','categories']]

In [5]:
df['title'] = preprocess_text(df, 'title')
df['abstract'] = preprocess_text(df, 'abstract')

##### For Unlabelled Data

In [6]:
df_unlab = unlabelled_data[['title', 'abstract']]

In [7]:
df_unlab['title'] = preprocess_text(df_unlab, 'title')
df_unlab['abstract'] = preprocess_text(df_unlab, 'abstract')

In [8]:
unlab_x = df_unlab[['title', 'abstract']].agg(' '.join, axis=1)

### Create Training Dataset

In [9]:
x = df[['title', 'abstract']].agg(' '.join, axis=1)

In [10]:
y = df["categories"]

In [11]:
from sklearn.model_selection import train_test_split as tts
train_x,test_x,train_y,test_y=tts(x,y,test_size=0.2,stratify=y)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(stop_words = 'english')
tfidf_train = tfidf_vect.fit_transform(train_x)
tfidf_test = tfidf_vect.transform(test_x)
tfidf_df = pd.DataFrame(tfidf_train.A, columns=tfidf_vect.get_feature_names())

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

lg = KNeighborsClassifier(n_neighbors=7)

lg.fit(unlab_train,unlab_y)
pred = lg.predict(tfidf_test)

print(accuracy_score(test_y,pred))

0.8123456790123457


In [14]:
def make_prediction(data):
    batch_vect = tfidf_vect.transform(data.to_numpy()).toarray()
    batch_prediction = lg.predict(batch_vect)
    prob_prediction = lg.predict_proba(batch_vect)
    
    return batch_prediction, prob_prediction

In [15]:
def batch_creator(df, batch_size):
    return (df[pos: pos + batch_size] for pos in range(0, len(df), batch_size))

In [16]:
unlab_df = pd.Series(unlab_x,
              name="text")

unlab_df = unlab_df.to_frame(name="text")

In [24]:
from scipy.stats import entropy
unlab_train = tfidf_vect.transform(unlab_df["text"])
unlab_y = lg.predict(unlab_train)
probs = lg.predict_proba(unlab_train)
unlab_entropy = np.apply_along_axis(entropy, 1, probs)

In [21]:
train_data = pd.DataFrame({"text": train_x, "categories": train_y})
train_data.reset_index(inplace=True, drop=True)

In [22]:
def active_learning_model(traindata):
    x = tfidf_vect.transform(traindata["text"])
    print(x.shape)
    y = traindata["categories"]
    
    active_lg = lg.fit(x, y)

    active_train_preds = lg.predict(x)
    active_preds = lg.predict(tfidf_test)
    
    train_active_acc = accuracy_score(y, active_train_preds)
    active_acc = accuracy_score(test_y, active_preds)
    return active_lg, active_acc, train_active_acc

In [23]:
batch_num = 0
batch_size=1000
for batch in batch_creator(unlab_df, batch_size):
    sampled_pool = pd.DataFrame()
    predictions, probabilities = make_prediction(batch["text"])

    for index, data in batch.iterrows(): 
        if(unlab_entropy[index] <= 0.80):
        #if(np.max(probabilities[index - batch_num * batch_size]) <= 0.30 ):
            sampled_pool = sampled_pool.append({
                'text': data['text'],
                'categories': predictions[index - batch_num * batch_size]
            }, ignore_index=True)     
    
    train_data = train_data.append(sampled_pool, ignore_index=True)
    active_model, active_score, train_active_score = active_learning_model(train_data)
    print("Testing score : ", active_score)
    print("Training score : ", train_active_score)
    print(train_data.shape)
    print("***********************")
    if batch_num == 50:
        break
        
    batch_num += 1

(7225, 23123)
Testing score :  0.8141975308641975
Training score :  0.8703114186851211
(7225, 2)
***********************
(7983, 23123)
Testing score :  0.8185185185185185
Training score :  0.8832519103094075
(7983, 2)
***********************
(8725, 23123)
Testing score :  0.8166666666666667
Training score :  0.8921489971346704
(8725, 2)
***********************
(9485, 23123)
Testing score :  0.817283950617284
Training score :  0.8985767000527148
(9485, 2)
***********************
(10230, 23123)
Testing score :  0.8160493827160494
Training score :  0.9058651026392962
(10230, 2)
***********************
(10970, 23123)
Testing score :  0.8166666666666667
Training score :  0.9103919781221513
(10970, 2)
***********************
(11699, 23123)
Testing score :  0.821604938271605
Training score :  0.9168304983331909
(11699, 2)
***********************
(12442, 23123)
Testing score :  0.8203703703703704
Training score :  0.9208326635589134
(12442, 2)
***********************
(13171, 23123)
Testing sco

KeyboardInterrupt: 